##   Feature selection Sequential

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from datetime import datetime

import sys
sys.path.insert(0, 'G:/work/GitHub/ml_baseline_things/functions/')
sys.path.insert(0, 'G:/work/GitHub/ml_baseline_things/featureSelectionExhaustiveAndSelection')


#from exhaustive_feature_selector import ExhaustiveFeatureSelector as EFS
from sequential_feature_selector import SequentialFeatureSelector as SFS

import CheckClassificationModels as ccm
import supportFunctions as sf
import featureEngineringFunctions as fef


In [3]:
#dataTrain, dataTrainTarget, dataTest, dataTrainScaled, dataTestScaled = sf.load_modified_data('data_modified/dataModified.pkl')
#len(dataTrain.columns)
featureListLElev = [
'AGE_lev_train',
'Age-Class_cnt_train',
'Deck_LE_train',
'FamilySize_cnt_train',
'Fare_lev_train',
'HasBloodRelatives_train',
'HasWifeHasb_train',
'IsAlone_train',
'Pclass_cnt_train',
'SEX_LE_train',
'Title_LE_train'
]
dataTrain = fef.create_dataset_from_features (featureListLElev, '2.1-featuresPack/')
dataTrainTarget =fef.create_dataset_from_features (['Survived_train'], '2.1-featuresPack/')

In [7]:
RS = 100
cv = StratifiedKFold(n_splits=4, random_state=RS, shuffle=True)

## Sequential

### Forward

In [9]:
#%%capture --no-display
model = LGBMClassifier(random_state=RS, n_jobs=-1,min_data=1, min_data_in_bin=1, min_child_samples=15)

sfs = SFS(model, 
           k_features='parsimonious', 
           forward=True, 
           floating=True, 
           verbose=0,
           scoring='accuracy',
           cv=2)

#sfs = sfs.fit(np.array(dataTrain), np.array(dataTrainTarget), custom_feature_names = tuple(dataTrain.columns))
sfs = sfs.fit(dataTrain, dataTrainTarget, custom_feature_names = tuple(dataTrain.columns))
sfs.subsets_

LightGBMError: Cannot construct Dataset since there are no useful features.
It should be at least two unique rows.
If the num_row (num_data) is small, you can set min_data=1 and min_data_in_bin=1 to fix this.
Otherwise, please make sure you are using the right dataset

In [6]:
pd.DataFrame.from_dict(sfs.get_metric_dict()).T

AttributeError: SequentialFeatureSelector has not been fitted, yet.

In [ ]:
# and the result is: 22 iter is best
selectedFeatures = list(sfs.subsets_[22]['feature_names'])
selectedFeatures

## Save to disk

In [ ]:
comment this line if ready
_ = joblib.dump(selectedFeatures, 
            open('selectedFeatures/selectedFeaturesLGBM--{}.pkl'.format(datetime.now().strftime("%Y-%m-%d--%H-%M")), 'wb'), 9)